<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong>
       Importando bibliotecas
    </strong>
</p>

In [1]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import numbers
import xlwings as xw


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Criando planilha de controle de entregas
    </strong>
</p>

In [2]:
controle_path = 'dados/controle_entregas.xlsx'

wb = Workbook() 

ws = wb.active

ws.title = 'Entregas'

ws.append([
    'Data',
    'Estado',
    'Motorista',
    'Entregas Previstas',
    'Entregas no Prazo',
    'Entregas Realizadas'
])

ws.append(['01/01/2026', 'RJ', 'Joaquim', '10', '6', '7'])
ws.append(['02/01/2026', 'SP', 'Athena',  '10', '2', '2'])
ws.append(['03/01/2026', 'MG', 'Tornado', '10', '8', '8'])
ws.append(['04/01/2026', 'SP', 'Fe Lelo', '10', '9', '9'])
ws.append(['05/01/2026', 'AC', 'Fafa',    '10', '8', '8'])
ws.append(['02/01/2026', 'RJ', 'Joaquim', '10', '4', '4'])
ws.append(['03/01/2026', 'SP', 'Athena',  '10', '2', '2'])
ws.append(['04/01/2026', 'MG', 'Tornado', '10', '1', '3'])
ws.append(['05/01/2026', 'SP', 'Fe Lelo', '10', '4', '8'])
ws.append(['06/01/2026', 'AC', 'Fafa',    '10', '2', '8'])

wb.save(controle_path)



<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Inserindo novos indicadores
    </strong>
</p>

In [3]:
wb = load_workbook(controle_path)
ws = wb.active
 
ws["G1"] = "Taxa de sucesso"
ws["H1"] = "Taxa de atraso"
 
for linha in range(2, ws.max_row + 1):
        
    ws[f'G{linha}'] = f'=F{linha}/D{linha}' #G2  = F2/D2 (TAXA DE SUCESSO)

    ws[f'H{linha}'] = f'=1-G{linha}'        #H2  = 1-G2 (TAXA DE ATRASO)

wb.save(controle_path)
print('Formulas inseridas com sucesso!')


Formulas inseridas com sucesso!


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Formatando as taxas como percentual
    </strong>
</p>

In [4]:
wb = load_workbook(controle_path)
ws = wb.active

for linha in range(2, ws.max_row + 1):

    ws[f'G{linha}'].number_format = "0.00%"
    ws[f'H{linha}'].number_format = "0.00%"

wb.save(controle_path)
print('Tipagem alterada com sucesso')

Tipagem alterada com sucesso


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Resumo gerencial por estado
    </strong>
</p>

In [5]:
from openpyxl import load_workbook

df = pd.read_excel(controle_path, sheet_name="Entregas")

# taxas
df['Taxa de sucesso 2'] = df['Entregas Realizadas'] / df['Entregas Previstas']
df['Taxa de atraso'] = 1 - df['Taxa de sucesso 2']

resumo_estado = df.groupby('Estado').agg({
    "Entregas Previstas": "sum",
    "Entregas Realizadas": "sum",
    "Taxa de sucesso 2": "mean",
    "Taxa de atraso": "mean"
})

wb = load_workbook(controle_path)

if 'Resumo' in wb.sheetnames:
    del wb['Resumo']


ws_resumo = wb.create_sheet("Resumo")

resumo_estado = resumo_estado.reset_index()

cabecalho = ["Estado", "Entregas Previstas", "Entregas Realizadas", "Taxa de sucesso 2", "Taxa de atraso"]

ws_resumo.append(cabecalho)

for index, row in resumo_estado.iterrows():
    ws_resumo.append([
        row["Estado"],
        row["Entregas Previstas"],
        row["Entregas Realizadas"],
        row["Taxa de sucesso 2"],
        row["Taxa de atraso"]
    ])

for linha in range(2, ws_resumo.max_row +1):
    ws_resumo[f'D{linha}'].number_format = '0.00%'
    ws_resumo[f'E{linha}'].number_format = '0.00%'


wb.save(controle_path)



<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Cálculos de taxas e inserindo no excel
    </strong>
</p>